### 路径

In [1]:
import sys
sys.path.append('/tmp-data/zhx/DriverOrderOfflineRL/tianshou')
sys.path.append('/tmp-data/zhx/DriverOrderOfflineRL/gym')

### import package

In [2]:
import argparse
import datetime
import os
import pickle
import pprint

import numpy as np
import torch
from torch import nn
import h5py
from torch.utils.tensorboard import SummaryWriter
from typing import Any, Dict, Optional, Sequence, Tuple, Union

from tianshou.data import Collector, VectorReplayBuffer, ReplayBuffer
from tianshou.policy import DiscreteCQLPolicy
from tianshou.trainer import offline_trainer
from tianshou.utils import TensorboardLogger, WandbLogger

/tmp-data/conda/anaconda3/envs/rl_test/lib/python3.9/site-packages/wandb/sdk/launch/builder/build.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/tmp-data/conda/anaconda3/envs/rl_test/lib/python3.9/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


### args config

In [64]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="OrderFilter")
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--eps-test", type=float, default=0.001)
    parser.add_argument("--lr", type=float, default=0.0001)
    parser.add_argument("--gamma", type=float, default=0.99)
    parser.add_argument("--num-quantiles", type=int, default=32)
    parser.add_argument("--n-step", type=int, default=1)
    parser.add_argument("--target-update-freq", type=int, default=500)
    parser.add_argument("--min-q-weight", type=float, default=10.)
    parser.add_argument("--epoch", type=int, default=100)
    parser.add_argument("--update-per-epoch", type=int, default=10000)
    parser.add_argument("--batch-size", type=int, default=32)
    parser.add_argument("--hidden-sizes", type=int, nargs="*", default=[512])
    parser.add_argument("--test-num", type=int, default=10)
    parser.add_argument("--frames-stack", type=int, default=1)
    parser.add_argument("--scale-obs", type=int, default=0)
    parser.add_argument("--logdir", type=str, default="log")
    parser.add_argument("--render", type=float, default=0.)
    parser.add_argument("--resume-path", type=str, default='/tmp-data/zhx/DriverOrderOfflineRL/scripts/log/OrderFilter/cql/0/231121-070714/1700.pth')
    parser.add_argument("--resume-id", type=str, default=None)
    parser.add_argument(
        "--logger",
        type=str,
        default="wandb",
        choices=["tensorboard", "wandb"],
    )
    parser.add_argument("--wandb-project", type=str, default="offline_driver.QRDQN")
    parser.add_argument(
        "--watch",
        default=False,
        action="store_true",
        help="watch the play of pre-trained policy only"
    )
    parser.add_argument("--log-interval", type=int, default=100)
    parser.add_argument(
        "--load-buffer-name", type=str, default="/tmp-data/yanhaoyue/workspace/RL/data/support_feature_buffer.h5"
    )
    parser.add_argument(
        "--buffer-from-rl-unplugged", action="store_true", default=True
    )
    parser.add_argument(
        "--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu"
    )
    args = parser.parse_known_args()[0]
    return args

In [65]:
args = get_args()
args

Namespace(task='OrderFilter', seed=0, eps_test=0.001, lr=0.0001, gamma=0.99, num_quantiles=32, n_step=1, target_update_freq=500, min_q_weight=10.0, epoch=100, update_per_epoch=10000, batch_size=32, hidden_sizes=[512], test_num=10, frames_stack=1, scale_obs=0, logdir='log', render=0.0, resume_path='/tmp-data/zhx/DriverOrderOfflineRL/scripts/log/OrderFilter/cql/0/231121-070714/1700.pth', resume_id=None, logger='wandb', wandb_project='offline_driver.QRDQN', watch=False, log_interval=100, load_buffer_name='/tmp-data/yanhaoyue/workspace/RL/data/support_feature_buffer.h5', buffer_from_rl_unplugged=True, device='cpu')

### load buffer

In [66]:
def load_buffer(buffer_path: str) -> ReplayBuffer:
    with h5py.File(buffer_path, "r") as dataset:
        buffer = ReplayBuffer.from_data(
            obs=dataset["observations"],
            act=dataset["actions"],
            rew=dataset["rewards"],
            done=dataset["terminals"],
            obs_next=dataset["next_observations"]
        )
    return buffer

In [67]:
buffer = load_buffer(args.load_buffer_name)

/tmp-data/zhx/DriverOrderOfflineRL/tianshou/tianshou/data/batch.py:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  obj_array = np.asanyarray(obj)


### network

In [68]:
class DQN(nn.Module):
    """Reference: Human-level control through deep reinforcement learning.

    For advanced usage (how to customize the network), please refer to
    :ref:`build_the_network`.
    """

    def __init__(
        self,
        state_shape: Sequence[int],
        action_shape: Sequence[int],
        device: Union[str, int, torch.device] = "cpu",
        features_only: bool = False,
        output_dim: Optional[int] = None,
    ) -> None:
        super().__init__()
        self.device = device
        self.net = nn.Sequential(
            nn.Linear(state_shape, 512), nn.ReLU(inplace=True),
            nn.Linear(512, action_shape), nn.ReLU(inplace=True),
            nn.Flatten()
        )
        with torch.no_grad():
            self.output_dim = np.prod(self.net(torch.zeros(1, state_shape)).shape[1:])
        if not features_only:
            self.net = nn.Sequential(
                self.net, nn.Linear(self.output_dim, 512), nn.ReLU(inplace=True),
                nn.Linear(512, np.prod(action_shape))
            )
            self.output_dim = np.prod(action_shape)
        elif output_dim is not None:
            self.net = nn.Sequential(
                self.net, nn.Linear(self.output_dim, output_dim),
                nn.ReLU(inplace=True)
            )
            self.output_dim = output_dim

    def forward(
        self,
        obs: Union[np.ndarray, torch.Tensor],
        state: Optional[Any] = None,
        info: Dict[str, Any] = {},
    ) -> Tuple[torch.Tensor, Any]:
        r"""Mapping: s -> Q(s, \*)."""
        obs = torch.as_tensor(obs, device=self.device, dtype=torch.float32)
        return self.net(obs), state
class QRDQN(DQN):
    """Reference: Distributional Reinforcement Learning with Quantile \
    Regression.

    For advanced usage (how to customize the network), please refer to
    :ref:`build_the_network`.
    """

    def __init__(
        self,
        state_shape: Sequence[int],
        action_shape: Sequence[int],
        num_quantiles: int = 32,
        device: Union[str, int, torch.device] = "cpu",
    ) -> None:
        self.action_num = np.prod(action_shape)
        super().__init__(state_shape, self.action_num * num_quantiles, device)
        self.num_quantiles = num_quantiles

    def forward(
        self,
        obs: Union[np.ndarray, torch.Tensor],
        state: Optional[Any] = None,
        info: Dict[str, Any] = {},
    ) -> Tuple[torch.Tensor, Any]:
        r"""Mapping: x -> Z(x, \*)."""
        obs, state = super().forward(obs)
        obs = obs.view(-1, self.action_num, self.num_quantiles)
        return obs, state

### seed

In [69]:
# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)

### buffer

In [70]:
# import pandas as pd
# buffer_data = pd.read_csv('/tmp-data/yanhaoyue/workspace/RL/data/support_feature_buffer.csv', sep='\t')

In [71]:
# import ast
# s = buffer_data['s'].apply(ast.literal_eval).values
# a = buffer_data['a'].values
# r = buffer_data['r'].values
# s_ = buffer_data['s_'].apply(ast.literal_eval).values
# d = buffer_data['d'].values

In [72]:
# s = np.array([np.array(item) for item in s])
# s_ = np.array([np.array(item) for item in s_])
# d = np.bool_(d)

In [73]:

# with h5py.File('/tmp-data/yanhaoyue/workspace/RL/data/support_feature_buffer.h5', 'w') as f:
#     f.create_dataset('observations', data=s)
#     f.create_dataset('actions', data=a)
#     f.create_dataset('rewards', data=r)
#     f.create_dataset('terminals', data=d)
#     f.create_dataset('next_observations', data=s_)

In [74]:
# buffer
if args.buffer_from_rl_unplugged:
    buffer = load_buffer(args.load_buffer_name)
else:
    assert os.path.exists(args.load_buffer_name), \
        "Please run atari_dqn.py first to get expert's data buffer."
    if args.load_buffer_name.endswith(".pkl"):
        buffer = pickle.load(open(args.load_buffer_name, "rb"))
    elif args.load_buffer_name.endswith(".hdf5"):
        buffer = VectorReplayBuffer.load_hdf5(args.load_buffer_name)
    else:
        print(f"Unknown buffer format: {args.load_buffer_name}")
        exit(0)
print("Replay buffer size:", len(buffer), flush=True)

Replay buffer size: 837295


### env

In [75]:
args.state_shape = buffer.obs.shape[1]
args.action_shape = 6 # 0， 1， 2， 3， 4， 5分位数
print("Observations shape:", args.state_shape)
print("Actions shape:", args.action_shape)

Observations shape: 109
Actions shape: 6


### model

In [76]:
# model
net = QRDQN(args.state_shape, args.action_shape, args.num_quantiles, args.device)
optim = torch.optim.Adam(net.parameters(), lr=args.lr)

### policy

In [77]:
# define policy
policy = DiscreteCQLPolicy(
    net,
    optim,
    args.gamma,
    args.num_quantiles,
    args.n_step,
    args.target_update_freq,
    min_q_weight=args.min_q_weight,
).to(args.device)

### load policy

In [78]:
# load a previous policy
if args.resume_path:
    policy.load_state_dict(torch.load(args.resume_path, map_location=args.device)['model'])
    print("Loaded agent from: ", args.resume_path)

Loaded agent from:  /tmp-data/zhx/DriverOrderOfflineRL/scripts/log/OrderFilter/cql/0/231121-070714/1700.pth


In [85]:
policy.model(torch.from_numpy(buffer[0]['obs']).view(1, -1))[0].sum(2).argmax(1)

tensor([5])

In [86]:
error = 0
for idx in range(1000, 1100):
    error += abs(policy.model(torch.from_numpy(buffer[idx]['obs']).view(1, -1))[0][0].sum(1).argmax().item() - buffer[idx]['act'])
    print('轮次', idx, 
          '模型选的动作',
          policy.model(torch.from_numpy(buffer[idx]['obs']).view(1, -1))[0][0].sum(1).argmax().item(), 
          '想要学习的动作',
          buffer[idx]['act'])
print('error', error / 100)

轮次 1000 模型选的动作 5 想要学习的动作 1.0
轮次 1001 模型选的动作 5 想要学习的动作 4.0
轮次 1002 模型选的动作 5 想要学习的动作 0.0
轮次 1003 模型选的动作 5 想要学习的动作 5.0
轮次 1004 模型选的动作 5 想要学习的动作 3.0
轮次 1005 模型选的动作 5 想要学习的动作 3.0
轮次 1006 模型选的动作 5 想要学习的动作 4.0
轮次 1007 模型选的动作 5 想要学习的动作 0.0
轮次 1008 模型选的动作 5 想要学习的动作 0.0
轮次 1009 模型选的动作 5 想要学习的动作 3.0
轮次 1010 模型选的动作 5 想要学习的动作 3.0
轮次 1011 模型选的动作 5 想要学习的动作 3.0
轮次 1012 模型选的动作 5 想要学习的动作 4.0
轮次 1013 模型选的动作 5 想要学习的动作 4.0
轮次 1014 模型选的动作 5 想要学习的动作 4.0
轮次 1015 模型选的动作 5 想要学习的动作 5.0
轮次 1016 模型选的动作 5 想要学习的动作 0.0
轮次 1017 模型选的动作 5 想要学习的动作 0.0
轮次 1018 模型选的动作 5 想要学习的动作 2.0
轮次 1019 模型选的动作 5 想要学习的动作 4.0
轮次 1020 模型选的动作 5 想要学习的动作 4.0
轮次 1021 模型选的动作 5 想要学习的动作 5.0
轮次 1022 模型选的动作 5 想要学习的动作 5.0
轮次 1023 模型选的动作 5 想要学习的动作 5.0
轮次 1024 模型选的动作 5 想要学习的动作 5.0
轮次 1025 模型选的动作 5 想要学习的动作 5.0
轮次 1026 模型选的动作 5 想要学习的动作 3.0
轮次 1027 模型选的动作 5 想要学习的动作 5.0
轮次 1028 模型选的动作 5 想要学习的动作 5.0
轮次 1029 模型选的动作 5 想要学习的动作 5.0
轮次 1030 模型选的动作 5 想要学习的动作 1.0
轮次 1031 模型选的动作 5 想要学习的动作 0.0
轮次 1032 模型选的动作 5 想要学习的动作 5.0
轮次 1033 模型选的动作 5 想要学习的动作 5.0
轮次 1034 模型选的动作

### logger

In [25]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
args.algo_name = "cql"
log_name = os.path.join(args.task, args.algo_name, str(args.seed), now)
log_path = os.path.join(args.logdir, log_name)

# logger
if args.logger == "wandb":
    logger = WandbLogger(
        save_interval=1,
        name=log_name.replace(os.path.sep, "__"),
        run_id=args.resume_id,
        config=args,
        project=args.wandb_project,
    )
writer = SummaryWriter(log_path)
writer.add_text("args", str(args))
if args.logger == "tensorboard":
    logger = TensorboardLogger(writer)
else:  # wandb
    logger.load(writer)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hongxi. Use `wandb login --relogin` to force relogin
/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/tmp-data/conda/anaconda3/envs/rl_test/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


### help function

In [26]:
def save_best_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, "policy.pth"))

def stop_fn(mean_rewards):
    return False

# watch agent's performance
def watch():
    print("Setup test envs ...")
    policy.eval()
    policy.set_eps(args.eps_test)
    test_envs.seed(args.seed)
    print("Testing agent ...")
    test_collector.reset()
    result = test_collector.collect(n_episode=args.test_num, render=args.render)
    pprint.pprint(result)
    rew = result["rews"].mean()
    print(f'Mean reward (over {result["n/ep"]} episodes): {rew}')
    
def save_checkpoint_fn(epoch, env_step, gradient_step):
    # see also: https://pytorch.org/tutorials/beginner/saving_loading_models.html
    ckpt_path = os.path.join(log_path, str(epoch) + ".pth")
    torch.save({"model": policy.state_dict()}, ckpt_path)
    return ckpt_path

### train core

In [ ]:
if args.watch:
    watch()
    exit(0)
import wandb
for epoch in range(1, args.epoch + 1):
    wandb.log(policy.update(0, buffer, batch_size=args.batch_size, repeat=1))
    save_checkpoint_fn(epoch, 0, 0)